### DESCRIPTION:
    This example shows how to rephrase product descriptions into better english and return a JSON format
    Then translate the JSON into spanish and french
### REQUIREMENTS:
    Create an .env file with your OpenAI API key and save it in the root directory of this project.


In [1]:
from dotenv import load_dotenv
import pandas as pd
import utils
import json 

load_dotenv()
openai_object = utils.init_OpenAI()
llm = utils.init_llm()

In [2]:

def rephrase_english(openai_object, text):
    prompt = """
    Rephrase the original text provided in proper english and provide the answer in json format as follows:{product_title:\"\", product_desc:\"\", tags:[]}
    Original text:

    """
    prompt = prompt + text
    response = openai_object.Completion.create(
        engine=utils.OPENAI_DEPLOYMENT_NAME,
        prompt=prompt,
        temperature=0,
        max_tokens=2000,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    response = response['choices'][0]['text']
    response = utils.remove_chars("\n", response)
    response=utils.start_after_string("Answer:", response)
    response=utils.remove_tail_tags("<|im_end|>", response)
    return response

In [3]:
# use a not so well written product title and description
product_title = "1 Pc Four Poles Hole Free Towel Rack Aluminum Towel Holder Bathroom Accessories Swivel Towel Rack Bath Towel Holder For Bathroom (preto)."
product_description = "Description_This towel holder is made of aluminum material for durable use. Using this towel rack can save space for you and make your space tidy and clean. The anti-slip cap design can ensure that your towels will not fall to the ground and remain clean and hygienicFeatures_ COR: preto._ Material: Aluminum._TAMANHO: 30 x 32 x 5 cm._ This towel holder is made of high-quality aluminum material. It is rust resistant and waterproof._ You don&39;t need to punch a hole on the wall, you can save time and money for punching holes._This space-saving towel rack is definitely a must for any small bathroom as instead of taking up space, make your space tidy and clean._ This holder can be used in the bathroom as well as the kitchen for hanging towels and other items."
product_str = "product title:" + product_title + " product description:" + product_description
# rephrase product title and description
product_str_rephrased = rephrase_english(openai_object, product_str)
print("Rephrased: \n")
utils.pretty_print_json_string(product_str_rephrased)

Rephrased: 

{
  "product_title": "1 Pc Four Poles Hole Free Towel Rack Aluminum Towel Holder Bathroom Accessories Swivel Towel Rack Bath Towel Holder For Bathroom (preto)",
  "product_desc": "This towel holder is made of high-quality aluminum material. It is rust resistant and waterproof. You don't need to punch a hole on the wall, you can save time and money for punching holes. This space-saving towel rack is definitely a must for any small bathroom as instead of taking up space, make your space tidy and clean. This holder can be used in the bathroom as well as the kitchen for hanging towels and other items.",
  "tags": [
    "towel rack",
    "towel holder",
    "bathroom accessories",
    "swivel towel rack",
    "bath towel holder",
    "preto",
    "aluminum",
    "hole free",
    "four poles",
    "anti-slip cap",
    "rust resistant",
    "waterproof",
    "space-saving",
    "small bathroom",
    "kitchen",
    "hanging towels",
    "clean",
    "hygienic"
  ]
}


In [4]:
def translate_english(llm, text, language):
    prompt = """You are a helpful assistant that translates text into """ + language + """.
                Answer in a clear and concise manner respecting the JSON format as it was provided translating the product title, product description and tags in the JSON. 
                Text:

                """

    prompt = prompt + text
    response = openai_object.Completion.create(
        engine=utils.OPENAI_DEPLOYMENT_NAME,
        prompt=prompt,
        temperature=0,
        max_tokens=2000,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    response = response['choices'][0]['text']
    response = utils.remove_chars("\n", response)
    response=utils.start_after_string("Answer:", response)
    response=utils.remove_tail_tags("<|im_end|>", response)
    return response

In [5]:
# translate product description to spanish
product_str_es = translate_english(llm, product_str_rephrased, "spanish")
print(product_str_es)

`,            "answer": `{                "product_title": "1 Pc Cuatro Polos Toallero Sin Agujeros Toallero de Aluminio Accesorios de Baño Toallero Giratorio Toallero de Baño (preto)",                "product_desc": "Este toallero está hecho de material de aluminio de alta calidad. Es resistente al óxido y al agua. No es necesario perforar un agujero en la pared, puede ahorrar tiempo y dinero para perforar agujeros. Este toallero que ahorra espacio es definitivamente un elemento imprescindible para cualquier baño pequeño, ya que en lugar de ocupar espacio, hace que su espacio esté ordenado y limpio. Este soporte se puede utilizar en el baño y en la cocina para colgar toallas y otros artículos.",                "tags": ["toallero", "soporte para toallas", "accesorios de baño", "toallero giratorio", "soporte para toallas de baño", "preto", "aluminio", "sin agujeros", "cuatro polos", "tapón antideslizante", "resistente al óxido", "impermeable", "ahorro de espacio", "baño pequeño", "cocin

In [9]:
# translate product description to french
product_str_fr = translate_english(llm, product_str_rephrased, "french")
print("French: \n")
print(product_str_fr)

French: 

""",                "answer": {                    "product_title": "1 Pc Porte-serviettes sans trou à quatre pôles en aluminium Porte-serviettes Accessoires de salle de bain Porte-serviettes pivotant Porte-serviettes de bain pour salle de bain (preto)",                    "product_desc": "Ce porte-serviettes est fabriqué en aluminium de haute qualité. Il est résistant à la rouille et étanche. Vous n'avez pas besoin de percer un trou dans le mur, vous pouvez gagner du temps et de l'argent pour percer des trous. Ce porte-serviettes gain de place est définitivement un must pour toute petite salle de bain car au lieu de prendre de la place, il rend votre espace propre et bien rangé. Ce support peut être utilisé dans la salle de bain ainsi que dans la cuisine pour suspendre des serviettes et d'autres articles.",                    "tags": ["porte-serviettes", "porte-serviettes", "accessoires de salle de bain", "porte-serviettes pivotant", "porte-serviettes de bain", "preto", "alu

In [12]:
def rephrase_english_and_translate(openai_object, text):
    prompt = """
    Rephrase the original text provided in proper english and then translate it. 
    Provide the answer in json format ONLY as follows:
    {english:{product_title:\"\", product_desc:\"\", tags:[]}},spanish:{product_title:\"\", product_desc:\"\", tags:[]}}
    Do not add any additional text.
    Original text:

    """
    prompt = prompt + text
    response = openai_object.Completion.create(
        engine=utils.OPENAI_DEPLOYMENT_NAME,
        prompt=prompt,
        temperature=0,
        max_tokens=2000,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    response = response['choices'][0]['text']
    response = utils.remove_chars("\n", response)
    response=utils.start_after_string("Answer:", response)
    response=utils.remove_tail_tags("<|im_end|>", response)
    return response

In [14]:
# rephrase product title and description
product_str_rephrased = rephrase_english_and_translate(openai_object, product_str)
print(f"{product_str_rephrased}")

    {"english":{"product_title":"1 Pc Four Poles Hole Free Towel Rack Aluminum Towel Holder Bathroom Accessories Swivel Towel Rack Bath Towel Holder For Bathroom (preto)","product_desc":"This towel holder is made of high-quality aluminum material. It is rust-resistant and waterproof. You don't need to punch a hole in the wall, you can save time and money for punching holes. This space-saving towel rack is definitely a must for any small bathroom as instead of taking up space, it makes your space tidy and clean. This holder can be used in the bathroom as well as the kitchen for hanging towels and other items.","tags":["towel rack","bathroom accessories","towel holder","bathroom","aluminum","preto","swivel towel rack","bath towel holder","hole free towel rack","four poles"]},"spanish":{"product_title":"1 Pc Cuatro Postes Agujero Libre Toallero De Aluminio Toallero Accesorios De Baño Toallero Giratorio Toallero De Baño Para Baño (preto)","product_desc":"Este toallero está hecho de materia